In [14]:
!pip install cryptocmd


                                              0.0/47.5 kB ? eta -:--:--
     ---------------------------------------- 47.5/47.5 kB 2.3 MB/s eta 0:00:00


In [87]:
# alpha vnatage API key: 53CG7RIYNCPMFA34
import requests
url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&RANGE=2022-07-01&RANGE=2023-08-31&apikey=53CG7RIYNCPMFA34&datatype=csv'
r = requests.get(url)
#data = r.json()

In [88]:
url = 'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=BTC&to_currency=USD&market=USD&RANGE=2022-07-01&RANGE=2023-08-31&apikey=53CG7RIYNCPMFA34&datatype=csv'


In [76]:
import pandas as pd
from pandas.io.json import json_normalize


test_df = pd.DataFrame.from_dict(data, orient = 'index')
test2_df = json_normalize(data)

C:\Users\corwi\AppData\Local\Temp\ipykernel_1820\2078720994.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  test2_df = json_normalize(data)


In [40]:
def flatten_nested_json_df(df):
    
    df = df.reset_index()
    
    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")
    
    
    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()
    
    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        
        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace
        
        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            # Prevent combinatorial explosion when multiple
            # cols have lists or lists of lists
            df = df.reset_index(drop=True)
            new_columns.append(col)
        
        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
        
        print(f"lists: {list_columns}, dicts: {dict_columns}")
        
    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

In [77]:
test_df.head()

,1. Information,2. Digital Currency Code,3. Digital Currency Name,4. Market Code,5. Market Name,6. Last Refreshed,7. Time Zone,2024-12-04,2024-12-03,2024-12-02,...,2023-12-30,2023-12-29,2023-12-28,2023-12-27,2023-12-26,2023-12-25,2023-12-24,2023-12-23,2023-12-22,2023-12-21
Meta Data,Daily Prices and Volumes for Digital Currency,BTC,Bitcoin,EUR,Euro,2024-12-04 00:00:00,UTC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Time Series (Digital Currency Daily),NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'1. open': '91249.72000000', '2. high': '9188...","{'1. open': '91279.95000000', '2. high': '9175...","{'1. open': '92096.25000000', '2. high': '9309...",...,"{'1. open': '38216.07000000', '2. high': '3868...","{'1. open': '38498.12000000', '2. high': '3906...","{'1. open': '39156.55000000', '2. high': '3944...","{'1. open': '38533.41000000', '2. high': '3938...","{'1. open': '39583.10000000', '2. high': '3959...","{'1. open': '39195.20000000', '2. high': '3986...","{'1. open': '39810.33000000', '2. high': '4000...","{'1. open': '39998.13000000', '2. high': '4001...","{'1. open': '39877.59000000', '2. high': '4039...","{'1. open': '39898.26000000', '2. high': '4026..."


In [78]:
test3 = flatten_nested_json_df(test_df.transpose())

original shape: (357, 3)
original columns: Index(['index', 'Meta Data', 'Time Series (Digital Currency Daily)'], dtype='object')
lists: [], dicts: []
final shape: (357, 3)
final columns: Index(['index', 'Meta Data', 'Time Series (Digital Currency Daily)'], dtype='object')


In [79]:
custom_col = list(test_df.columns)
daily_dat = list(test_df.iloc[1,:])

print(daily_dat)
print(custom_col)



[nan, nan, nan, nan, nan, nan, nan, {'1. open': '91249.72000000', '2. high': '91880.00000000', '3. low': '90935.75000000', '4. close': '91778.68000000', '5. volume': '14.68991966'}, {'1. open': '91279.95000000', '2. high': '91751.07000000', '3. low': '88886.03000000', '4. close': '91229.73000000', '5. volume': '378.30146135'}, {'1. open': '92096.25000000', '2. high': '93098.99000000', '3. low': '90011.05000000', '4. close': '91275.31000000', '5. volume': '371.87919391'}, {'1. open': '91143.07000000', '2. high': '92599.99000000', '3. low': '90470.00000000', '4. close': '92099.58000000', '5. volume': '195.27489538'}, {'1. open': '92138.65000000', '2. high': '92154.35000000', '3. low': '90722.66000000', '4. close': '91143.29000000', '5. volume': '147.37387973'}, {'1. open': '90609.11000000', '2. high': '93535.47000000', '3. low': '90296.09000000', '4. close': '92147.09000000', '5. volume': '370.72697579'}, {'1. open': '90831.46000000', '2. high': '91601.62000000', '3. low': '89731.4400000

In [80]:
print(pd.json_normalize(daily_dat[6]))

NotImplementedError: 

In [81]:
clean_btc_data = pd.DataFrame()
for j in range(0,len(daily_dat)):
    if j <5:
        continue

    date = pd.DataFrame([custom_col[j]])
    data = pd.json_normalize(daily_dat[j])

    full = pd.concat([date,data], axis = 1)
    clean_btc_data = pd.concat([clean_btc_data, full], axis = 0)







NotImplementedError: 

In [67]:
clean_btc_data.shape
clean_btc_data.head()

,0,1. open,2. high,3. low,4. close,5. volume
0,2024-12-03,227.2400,229.1100,226.6700,229.0000,3163815
0,2024-12-02,227.5000,228.3800,225.5100,227.3900,2656181
0,2024-11-29,227.7500,230.3600,227.1900,227.4100,2640253
0,2024-11-27,228.8300,229.1900,224.2700,226.9200,2995121
0,2024-11-26,226.7300,228.9800,225.5115,228.8300,4449543
